In [1]:
import string

In [2]:
class Enigma:
  def __init__(self):
    """ A -> B, B -> A ... Y -> Z, Z -> Y """
    plugboard_mapping = {}
    for i in range(0, 26, 2):
      plugboard_mapping[string.ascii_uppercase[i]] = string.ascii_uppercase[i+1]
      plugboard_mapping[string.ascii_uppercase[i+1]] = string.ascii_uppercase[i]

    self.rotors = [Rotor(i, 0) for i in range(3)]
    self.reflector = Reflector(1)
    self.plugboard = Plugboard(plugboard_mapping)

  def reset(self):
    for rotor in self.rotors:
      rotor.reset()

  def encode(self, plaintext):
    plaintext = plaintext.upper()
    if not plaintext.isalpha():
      print("Invalid plaintext!")
      pass
    
    ciphertext = ""
    for plaintext_char in plaintext:
      """ Normal Step """
      for i, rotor in enumerate(self.rotors):
        print(i)
        if (not rotor.rotate()):
          break

      """ Double Step """
      for turnover in self.rotors[1].turnovers:
        """ gets the subsequent character mod the alphabet """
        notch = string.ascii_uppercase[(string.ascii_uppercase.index(turnover) + 1) % 26]
        if string.ascii_uppercase[self.rotors[1].number_of_rotates % 26] == notch:
          print('1')
          if(self.rotors[1].rotate()):
            print('2')
            self.rotors[2].rotate()

      index = self.plugboard.forward(plaintext_char)

      for rotor in self.rotors:
        index = rotor.forward(index)

      index = self.reflector.forward(index)

      for rotor in self.rotors:
        index = rotor.reverse(index)

      ciphertext_char = self.plugboard.reverse(index)
      ciphertext += ciphertext_char

    return ciphertext

class Rotor:
  def __init__(self, rotor_number, ring_offset):
    self.rotor_number = rotor_number
    self.settings = [
      { "sequence": "EKMFLGDQVZNTOWYHXUSPAIBRCJ", "turnovers": ["R"]},
      { "sequence": "AJDKSIRUXBLHWTMCQGZNPYFVOE", "turnovers": ["F"]},
      { "sequence": "BDFHJLCPRTXVZNYEIWGAKMUSQO", "turnovers": ["W"]},
      { "sequence": "ESOVPZJAYQUIRHXLNFTGKDCMWB", "turnovers": ["K"]},
      { "sequence": "VZBRGITYUPSDNHLXAWMJQOFECK", "turnovers": ["A"]},
      { "sequence": "JPGVOUMFYQBENHZRDKASXLICTW", "turnovers": ["A", "N"]},
      { "sequence": "NZJHGRCXMYSWBOUFAIVLPEKQDT", "turnovers": ["A", "N"]},
      { "sequence": "FKQHTLXOCBJSPDZRAMEWNIUYGV", "turnovers": ["A", "N"]},
    ]
    self.sequence = self.settings[self.rotor_number]['sequence']
    self.turnovers = self.settings[self.rotor_number]['turnovers']
    self.ring_offset = ring_offset
    self.number_of_rotates = 0

    for _ in range(self.ring_offset):
      self.rotate()

  def reset(self):
    self.number_of_rotates = 0
    self.sequence = self.settings[self.rotor_number]['sequence']
    self.turnovers = self.settings[self.rotor_number]['turnovers']

    for _ in range(self.ring_offset):
      self.rotate()

  def forward(self, index):
    """ Returns the index of the character, in respect to the alphabet, after passing through the sequence """
    return string.ascii_uppercase.index(self.sequence[index])

  def reverse(self, index):
    """ Returns the index of the character, in respect to the sequence, after passing through the alphabet """
    return self.sequence.index(string.ascii_uppercase[index])

  def rotate(self):
    self.number_of_rotates += 1
    self.sequence = self.sequence[1:] + self.sequence[:1]
    for char in self.turnovers:
      if char == string.ascii_uppercase[self.number_of_rotates % 26]:
        return True
    return False

class Reflector:
  def __init__(self, reflector_number):
    settings = [
      "EJMZALYXVBWFCRQUONTSPIKHGD",
      "YRUHQSLDPXNGOKMIEBFZCWVJAT",
      "FVPJIAOYEDRZXWGCTKUQSBNMHL",
    ]
    self.sequence = settings[reflector_number]

  def forward(self, index):
    """ Returns the index of the character, in respect to the sequence, after passing through the alphabet """
    return self.sequence.index(string.ascii_uppercase[index])

class Plugboard:
  def __init__(self, mapping):
    self.mapping = mapping

  def forward(self, char):
    """ Returns the index of the mapped character """
    return string.ascii_uppercase.index(self.mapping[char])

  def reverse(self, index):
    """ Returns the character of the index """
    return self.mapping[string.ascii_uppercase[index]]

In [3]:
enigma = Enigma()

In [4]:
enigma.encode('EVAN')

0
0
0
0


'VQPD'

In [5]:
enigma.reset()
enigma.encode('l')

0


'M'